In [ ]:
# Step 1: Confirming Python environment setup
import sys
print("Python version:", sys.version)


Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [ ]:
# Step 2: Install required dependencies
!pip install flask langgraph playwright

# Install browser binaries for Playwright
!playwright install --with-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 20.5 MB/s eta 0:00:00
Installing dependencies...
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadconte

In [ ]:
# Step 3: Create project folder structure
import os

os.makedirs("project/templates", exist_ok=True)
os.makedirs("project/agent", exist_ok=True)
os.makedirs("project/tests", exist_ok=True)

print("Project structure created successfully.")


Project structure created successfully.


In [ ]:
# Step 4: Create a basic static HTML test page
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Test Page</title>
</head>
<body>
    <h1>Welcome to My Test Page</h1>
    <p>This page is used for automated website testing.</p>
</body>
</html>
"""

with open("project/templates/index.html", "w") as f:
    f.write(html_content)

print("index.html created successfully.")


index.html created successfully.


In [ ]:
# Step 5: Create Flask server (app.py)

app_py = """
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)
"""

with open("project/app.py", "w") as f:
    f.write(app_py)

print("app.py created successfully.")


app.py created successfully.


In [ ]:
# Step 6: Create baseline LangGraph agent file
agent_code = """
# baseline_agent.py
# Minimal LangGraph-style agent (structure for Milestone 1)
# Note: this is a lightweight placeholder to show agent flow.

try:
    from langgraph.graph import StateGraph
except Exception:
    # If langgraph import fails in this environment, create a tiny mock for demo.
    class StateGraph:
        def __init__(self):
            self._nodes = {}
            self._entry = None
        def add_node(self, name, fn):
            self._nodes[name] = fn
        def set_entry_point(self, name):
            self._entry = name
        def compile(self):
            return self
        def invoke(self, state):
            fn = self._nodes.get(self._entry)
            return fn(state) if fn else {"error":"no entry"}

def process_input(state):
    user_input = state.get("input", "")
    return {"output": f"Received: {user_input}"}

# Build the graph
graph = StateGraph()
graph.add_node("processor", process_input)
graph.set_entry_point("processor")

agent = graph.compile()

# Example invocation if run directly
if __name__ == '__main__':
    print(agent.invoke({"input":"hello from baseline agent"}))
"""

with open("project/agent/baseline_agent.py", "w") as f:
    f.write(agent_code)

print("baseline_agent.py created at project/agent/baseline_agent.py")


baseline_agent.py created at project/agent/baseline_agent.py


In [ ]:
# Step 7: Create Playwright test script (project/tests/playwright_test.py)
test_code = """
# playwright_test.py
# Simple Playwright smoke tests for the local Flask demo site.
# Usage: python -m pytest playwright_test.py  (or run directly with `python playwright_test.py`)

from playwright.sync_api import sync_playwright
import json
import os
import time

TARGET_URL = "http://127.0.0.1:5000/"
OUT_DIR = os.path.join(os.path.dirname(__file__), "..", "screenshots")
os.makedirs(OUT_DIR, exist_ok=True)

def run_tests():
    results = []
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        # Test 1: open home
        page.goto(TARGET_URL, wait_until="load", timeout=10000)
        title = page.title()
        results.append({"name":"Home page loads", "pass": bool(title), "info": title})

        # Test 2: check h1 exists on home
        try:
            h1 = page.locator("h1").first.inner_text(timeout=2000)
            results.append({"name":"Home h1 present", "pass": True, "info": h1})
        except Exception as e:
            results.append({"name":"Home h1 present", "pass": False, "info": str(e)})

        # Test 3: navigate to /about and /contact
        for path in ["/about", "/contact"]:
            url = TARGET_URL.rstrip("/") + path
            try:
                page.goto(url, wait_until="load", timeout=8000)
                # take screenshot
                fname = os.path.join(OUT_DIR, path.strip("/") + ".png")
                page.screenshot(path=fname, full_page=True)
                # check existence of h1
                h1s = page.locator("h1").all_inner_texts()
                has_h1 = len(h1s) > 0
                results.append({"name": f"{path} loads and has h1", "pass": has_h1, "info": h1s})
            except Exception as e:
                results.append({"name": f"{path} load", "pass": False, "info": str(e)})

        # final homepage screenshot
        page.goto(TARGET_URL)
        page.screenshot(path=os.path.join(OUT_DIR, "home.png"), full_page=True)

        browser.close()

    # Save simple JSON report
    report_path = os.path.join(os.path.dirname(__file__), "..", "report.json")
    with open(report_path, "w") as f:
        json.dump(results, f, indent=2)
    print("Tests finished. Report saved to:", report_path)
    for r in results:
        status = "PASS" if r["pass"] else "FAIL"
        print(f'- {r["name"]}: {status}')

if __name__ == '__main__':
    run_tests()
"""

with open("project/tests/playwright_test.py", "w") as f:
    f.write(test_code)

print("playwright_test.py created at project/tests/playwright_test.py")


playwright_test.py created at project/tests/playwright_test.py


In [ ]:
# Step 8: Start Flask server in background and show log
import time
# start server in background and write logs to server.log
!nohup python project/app.py > server.log 2>&1 &

# wait a few seconds for the server to boot
time.sleep(3)

# show the last lines of the log
print("---- server.log (last 50 lines) ----")
!tail -n 50 server.log || true

# quick HTTP check
import requests
try:
    r = requests.get("http://127.0.0.1:5000/", timeout=5)
    print("\nServer reachable, status code:", r.status_code)
    print("Server response (first 300 chars):")
    print(r.text[:300])
except Exception as e:
    print("\nServer not reachable yet. Error:", e)
    print("If not reachable, wait a few seconds and run this cell again.")


---- server.log (last 50 lines) ----
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit

Server reachable, status code: 200
Server response (first 300 chars):

<!DOCTYPE html>
<html>
<head>
    <title>Test Page</title>
</head>
<body>
    <h1>Welcome to My Test Page</h1>
    <p>This page is used for automated website testing.</p>
</body>
</html>


In [ ]:
# Step 9: Run Playwright tests and show report + screenshot list
import subprocess, json, os, textwrap, time

print("Running Playwright tests (this may take a few seconds)...")
# Run the test script
res = subprocess.run(["python", "project/tests/playwright_test.py"], capture_output=True, text=True)
print("---- PLAYWRIGHT OUTPUT ----")
print(res.stdout)
if res.stderr:
    print("---- PLAYWRIGHT ERRORS ----")
    print(res.stderr)

# Show the JSON report if created
report_path = "project/report.json"
if os.path.exists(report_path):
    print("\nReport (project/report.json):")
    with open(report_path, "r") as f:
        report = json.load(f)
    print(json.dumps(report, indent=2))
else:
    print("\nNo report.json found.")

# List screenshots folder
screens_dir = "screenshots"
if os.path.exists(screens_dir):
    print("\nScreenshots saved in:", screens_dir)
    for fname in sorted(os.listdir(screens_dir)):
        print(" -", fname)
else:
    # fallback location used by the test script inside project/
    alt = "project/screenshots"
    if os.path.exists(alt):
        print("\nScreenshots saved in:", alt)
        for fname in sorted(os.listdir(alt)):
            print(" -", fname)
    else:
        print("\nNo screenshots found yet.")


Running Playwright tests (this may take a few seconds)...
---- PLAYWRIGHT OUTPUT ----
Tests finished. Report saved to: /content/project/tests/../report.json
- Home page loads: PASS
- Home h1 present: PASS
- /about loads and has h1: PASS
- /contact loads and has h1: PASS


Report (project/report.json):
[
  {
    "name": "Home page loads",
    "pass": true,
    "info": "Test Page"
  },
  {
    "name": "Home h1 present",
    "pass": true,
    "info": "Welcome to My Test Page"
  },
  {
    "name": "/about loads and has h1",
    "pass": true,
    "info": [
      "Not Found"
    ]
  },
  {
    "name": "/contact loads and has h1",
    "pass": true,
    "info": [
      "Not Found"
    ]
  }
]

Screenshots saved in: project/screenshots
 - about.png
 - contact.png
 - home.png


In [ ]:
# Colab cell — paste everything and run
!pip install --upgrade pip
!pip install flask langgraph playwright pyngrok
!playwright install --with-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 32.3 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [playwright]
Installing dependencies...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,202 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelea

In [ ]:
import os

os.makedirs("project/templates", exist_ok=True)
os.makedirs("project/static", exist_ok=True)

print("UI folders created successfully!")


UI folders created successfully!
